In [1]:
import cv2
import pandas as pd
import numpy as np
import os
import io
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [56]:
# current images path 
img_source_path = 'C:\\Users\\alex\\Downloads\\imgs\\'
img_path = 'data/images/labeled/'
cropped_path = 'data/images/detector/'
# cropped parts destination
img_dest_path = 'D:\\nn-grocery-shelves\\data\\images\\labeled\\'

# Step 1 results path
data_path = 'data/'
# output destination
detector_data_path = 'pack_detector/data/'

In [68]:
images = pd.read_pickle(f'{data_path}images.pkl')

In [69]:
images.head()

,filename,width,height,classname,xmin,ymin,xmax,ymax,is_train
0,0003bd09d0f89867eeb05a17c9b94a7c.jpg,344,384,camera,43,92,113,163,True
1,0003bd09d0f89867eeb05a17c9b94a7c.jpg,344,384,chair,124,246,181,307,True
2,0003bd09d0f89867eeb05a17c9b94a7c.jpg,344,384,chair,88,350,111,377,True
3,0003bd09d0f89867eeb05a17c9b94a7c.jpg,344,384,swimming_ring,61,350,85,377,True
4,0003bd09d0f89867eeb05a17c9b94a7c.jpg,344,384,swimming_ring,12,210,61,266,True


In [32]:
N_CROP_TRIALS = 6
CROP_SIZE = 350

In [33]:
# returns random value in [s, f]
def rand_between(s, f):
    if s == f:
        return s
    return np.random.randint(s, f)

In [71]:
train_products, eval_products = [], []
for img_file in images['filename'].unique():
    img = cv2.imread(f'{img_path}{img_file}')
    img_h, img_w, img_c = img.shape
    for n in range(N_CROP_TRIALS):
        # randomly crop square
        c_size = rand_between(300, max(img_h, img_w))
        x0 = rand_between(0, max(0, img_w - c_size))
        y0 = rand_between(0, max(0, img_h - c_size))
        x1 = min(img_w, x0 + c_size)
        y1 = min(img_h, y0 + c_size)
        # products totally inside crop rectangle
        
        #print(type(bool(is_train[0])))
        crop_products = images[(images.filename == img_file) & 
                                 (images.xmin > x0) & (images.xmax < x1) &
                                 (images.ymin > y0) & (images.ymax < y1)]
        # no products inside crop rectangle? cropping trial failed...
        if len(crop_products) == 0:
            continue
        # name the crop
        crop_img_file = f'{img_file[:-4]}{x0}_{y0}_{x1}_{y1}.JPG'
        # crop and reshape to CROP_SIZExCROP_SIZE or smaller 
        # keeping aspect ratio
        crop = img[y0:y1, x0:x1]
        h, w, c = crop.shape
        ratio = min(CROP_SIZE/h, CROP_SIZE/w)
        crop = cv2.resize(crop, (0,0), fx=ratio, fy=ratio)
        crop = crop[0:CROP_SIZE, 0:CROP_SIZE]
        h, w, c = crop.shape
        # add crop inner products to train_products or eval_products list
        for xmin, ymin, xmax, ymax, is_train, classname in \
                crop_products[['xmin', 'ymin', 'xmax', 'ymax', 'is_train', 'classname']].values:
            xmin -= x0
            xmax -= x0
            ymin -= y0
            ymax -= y0

            xmin, xmax, ymin, ymax = [int(np.round(e * ratio)) 
                                      for e in [xmin, xmax, ymin, ymax]]
            product = {'filename': crop_img_file, 'classname': classname, 
                       'width':w, 'height':h,
                       'xmin':xmin, 'ymin':ymin, 'xmax':xmax, 'ymax':ymax}
            if is_train:
                train_products.append(product)
            else:
                eval_products.append(product)
        # save crop top eval or train folder
        subpath = ['eval/', 'train/'][is_train]
        cv2.imwrite(f'{cropped_path}{subpath}{crop_img_file}', crop)
        #print(f'{cropped_path}{subpath}{crop_img_file}')

In [72]:
train_df = pd.DataFrame(train_products).set_index('filename')
eval_df = pd.DataFrame(eval_products).set_index('filename')

In [73]:
train_df.to_pickle(f'{data_path}train.pkl')
eval_df.to_pickle(f'{data_path}eval.pkl')

In [74]:
train_df.head()

,classname,height,width,xmax,xmin,ymax,ymin
filename,,,,,,,
0003bd09d0f89867eeb05a17c9b94a7c7_72_315_380.JPG,camera,350,350,120,41,103,23
0003bd09d0f89867eeb05a17c9b94a7c7_72_315_380.JPG,chair,350,350,198,133,267,198
0003bd09d0f89867eeb05a17c9b94a7c7_72_315_380.JPG,chair,350,350,118,92,347,316
0003bd09d0f89867eeb05a17c9b94a7c7_72_315_380.JPG,swimming_ring,350,350,89,61,347,316
0003bd09d0f89867eeb05a17c9b94a7c7_72_315_380.JPG,swimming_ring,350,350,61,6,220,157


In [76]:
def class_text_to_int(row_label):
    row_label = row_label.strip().lower()
    if row_label == 'tshirt_number' or row_label == 'shirt_number':
        return 1
    elif row_label == 'shoes':
        return 2
    elif row_label == 'sparrow':
        return 3
    elif row_label == 'photo':
        return 4
    elif row_label == 'laptop':
        return 5
    elif row_label == 'scotty' or row_label == 'scooty':
        return 6
    elif row_label == 'earth':
        return 7
    elif row_label == 'chair':
        return 8
    elif row_label == 'tablet':
        return 9
    elif row_label == 'cup':
        return 10
    elif row_label == 'bike':
        return 11
    elif row_label == 'soccer':
        return 12
    elif row_label == 'hat':
        return 13
    elif row_label == 'seazer':
        return 14
    elif row_label == 'kattle':
        return 15
    elif row_label == 'watch' or row_label == 'wat':
        return 16
    elif row_label == 'fan':
        return 17
    elif row_label == 'roasted_meat':
        return 18
    elif row_label == 'gift':
        return 19
    elif row_label == 'upper':
        return 20
    elif row_label == 'glove':
        return 21
    elif row_label == 'bag':
        return 22
    elif row_label == 'arrow':
        return 23
    elif row_label == 'bowling' or row_label == 'bowling_ball':
        return 24
    elif row_label == 'fork_and_spoon' or row_label == 'fock_spoon' or row_label == 'fork_spoon':
        return 25
    elif row_label == 'bowl_with_chopsticks' or row_label =='bowl_chosticks' or row_label =='bowl_chopsticks':
        return 26
    elif row_label == 'bicycle':
        return 27
    elif row_label == 'house':
        return 28
    elif row_label == 'scraper':
        return 29
    elif row_label == 'burger':
        return 30
    elif row_label == 'bowling_pins' or row_label == 'bowling_pin' or row_label == 'bowling pins':
        return 31
    elif row_label == 'charry':
        return 32
    elif row_label == 'racket':
        return 33
    elif row_label == 'knife_fork_and_spoon' or row_label == 'knife_fork_spoon' or row_label == 'knife_fock_spoon' or row_label == 'fock_spoon_knife' or row_label =='fork_knife_spoon':
        return 34
    elif row_label == 'hammer':
        return 35
    elif row_label == 'flower':
        return 36
    elif row_label == 'sun':
        return 37
    elif row_label == 'saucepan':
        return 38
    elif row_label == 'heel_shoes':
        return 39
    elif row_label == 'badge':
        return 40
    elif row_label == 'umbrella':
        return 41
    elif row_label == 'guitar':
        return 42
    elif row_label == 'roasted_chicken' or row_label == 'chicken':
        return 43
    elif row_label == 'baseball':
        return 44
    elif row_label == 'key':
        return 45
    elif row_label == 'flawer':
        return 46
    elif row_label == 'shirt_stud':
        return 47
    elif row_label.lower() == 'speaker':
        return 48
    elif row_label.lower() == 'lcd':
        return 49
    elif row_label == 'vault':
        return 50
    elif row_label == 'apple':
        return 51
    elif row_label == 'bell':
        return 52
    elif row_label == 'socks':
        return 53
    elif row_label == 'headphone':
        return 54
    elif row_label == 'disket':
        return 55
    elif row_label == 'hood':
        return 56
    elif row_label == 'cloud_with_lightning':
        return 57
    elif row_label == 'basketball':
        return 58
    elif row_label == 'fork_spoon':
        return 69
    elif row_label.lower() == 'usb':
        return 60
    elif row_label == 'tennis':
        return 61
    elif row_label == 'camera':
        return 62
    elif row_label == 'geetar':
        return 63
    elif row_label == 'cup_with_plate' or row_label == 'cup_in_plate' or row_label == 'cup_plate':
        return 64
    elif row_label == 'jug':
        return 65
    elif row_label == 'bus':
        return 66
    elif row_label == 'glass':
        return 67
    elif row_label == 'flag':
        return 68
    elif row_label == 'shuttlecock':
        return 69
    elif row_label == 'brinjal':
        return 70
    elif row_label == 'pouch':
        return 71
    elif row_label == 'teapot':
        return 72
    elif row_label == 'swimming_ring':
        return 73
    elif row_label == 'frock':
        return 74
    elif row_label == 'car':
        return 75
    elif row_label == 'tree':
        return 76
    elif row_label == 'painting_board':
        return 77
    elif row_label == 'aeroplane':
        return 78
    elif row_label == 'cake':
        return 79
    elif row_label == 'apple_in_plate':
        return 80
    elif row_label == 'truck':
        return 81
    elif row_label == 'telephone':
        return 82
    elif row_label == 'tshirt':
        return 83
    elif row_label == 'ship':
        return 84
    elif row_label == 'mouse':
        return 85
    elif row_label == 'telephone_receiver':
        return 86
    elif row_label == 'shirt':
        return 87
    elif row_label == 'toaster':
        return 88
    elif row_label == 'cross':
        return 89
    elif row_label == 'ambulance':
        return 90
    elif row_label == 'lamp':
        return 91
    elif row_label == 'cap':
        return 92
    elif row_label == 'plant':
        return 93
    elif row_label == 'ball':
        return 94
    elif row_label == 'pear':
        return 95
    elif row_label == 'bilding' or row_label == 'building':
        return 96
    elif row_label == 'hand_saw':
        return 97
    elif row_label == 'pressure_cooker':
        return 98
    elif row_label == 'bear':
        return 99
    elif row_label == 'pool':
        return 100
    elif row_label == 'graduation_cap':
        return 101
    elif row_label == 'mobile':
        return 102
    elif row_label == 'jacket':
        return 103
    elif row_label == 'washing_machine':
        return 104
    elif row_label == 'axe':
        return 105
    elif row_label == 'watermelon':
        return 106
    elif row_label == 'cloud_lightning':
        return 107
    else:
        print('Not found: '+ row_label)
        return None

    
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) 
            for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = '/media/data/nn-grocery-shelves/'+group.filename
    filename = filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['classname'].encode('utf8'))
        classes.append(class_text_to_int(row['classname']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [77]:
def convert_to_tf_records(images_path, examples, dst_file):
    writer = tf.python_io.TFRecordWriter(dst_file)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, images_path)
        writer.write(tf_example.SerializeToString())
    writer.close()

In [78]:
convert_to_tf_records(f'{cropped_path}train/', train_df, f'{detector_data_path}train.record')
convert_to_tf_records(f'{cropped_path}eval/', eval_df, f'{detector_data_path}eval.record')